In [1]:
import csv
import pandas as pd
import sys
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
import numpy as np


[nltk_data] Downloading package stopwords to C:\Users\ABHINEET
[nltk_data]     SINGH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [3]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

In [4]:
max_features = 10000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 125
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
#print(len(x_train), 'train sequences')
#print(len(x_test), 'test sequences')
#print(x_test[0])
#print(y_test[0])

In [5]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
#print(x_test[0])

Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [6]:
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [7]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [8]:
df_csv = pd.read_csv('neg_rev.csv')


In [9]:
n_reviews = df_csv.iloc[:5001,0:2]
n_reviews = n_reviews.assign(rating = 0.0)


In [10]:
n_reviews
#n_reviews.iloc[4900:5002,0:2]
print((n_reviews.text[0]))
print(len(n_reviews.text[0]))

it worked for the first week then it only charge my phone to 20%. it is a waste of money.
89


In [11]:
df_csv1 = pd.read_csv('pos_rev.csv')

In [12]:
p_reviews = df_csv1.iloc[:5001,0:2]
p_reviews = p_reviews.assign(rating = 1.0)


In [13]:
p_reviews

,text,rating
0,These stickers work like the review says they ...,1.0
1,These are awesome and make my phone look so st...,1.0
2,"awesome! stays on, and looks great. can be use...",1.0
3,Came just as described.. It doesn't come unstu...,1.0
4,"Good case, solid build. Protects phone all aro...",1.0
...,...,...
4996,Better price and a little more milliamps (1500...,1.0
4997,worked great in two of my friends Nexus S 4G's...,1.0
4998,My original phone battery was losing memory so...,1.0
4999,TOOK ABOUT 4 DAYS TO GET HERE. WORKED AS EXPEC...,1.0


In [14]:
#frames = [n_reviews, p_reviews]
#union = pd.concat([df1, df2], ignore_index=True)


result = pd.concat([n_reviews, p_reviews], ignore_index=True)


In [15]:
print(len(result.text))

10002


In [223]:
print(result)
print(type(result))

                                                    text  rating
0      it worked for the first week then it only char...     0.0
1      It worked great for the first couple of weeks ...     0.0
2      Only works one side at a time. When you connec...     0.0
3      I bought 2 of this and tried to test first ......     0.0
4      This case for some reason is peeling, there is...     0.0
...                                                  ...     ...
9997   Better price and a little more milliamps (1500...     1.0
9998   worked great in two of my friends Nexus S 4G's...     1.0
9999   My original phone battery was losing memory so...     1.0
10000  TOOK ABOUT 4 DAYS TO GET HERE. WORKED AS EXPEC...     1.0
10001  I was skeptical when buying this product due t...     1.0

[10002 rows x 2 columns]
<class 'pandas.core.frame.DataFrame'>


In [16]:
def remove_stopwords(input_text):
    '''
    Function to remove English stopwords from a Pandas Series.
    
    Parameters:
        input_text : text to clean
    Output:
        cleaned Pandas Series 
    '''
    stopwords_list = stopwords.words('english')
    stop = ['.', ',' ,'!' ,'"','~']
    #print(stopwords_list)
    # Some words which might indicate a certain sentiment are kept via a whitelist
    whitelist = ["n't", "not", "no"]
    words = input_text.split() 
    clean_words = [word for word in words if (word not in stopwords_list and word not in stopwords_list or word in whitelist) and len(word) > 1] 
    return " ".join(clean_words) 
    
def remove_mentions(input_text):
    '''
    Function to remove mentions, preceded by @, in a Pandas Series
    
    Parameters:
        input_text : text to clean
    Output:
        cleaned Pandas Series 
    '''
    return re.sub(r'@\w+', '', input_text)

In [17]:
print((result.text[1]))

It worked great for the first couple of weeks then it just stopped completely.. so basically a small waste of money.


In [18]:
result = result[result['text'].notnull()]
result.text = result.text.apply(remove_stopwords).apply(remove_mentions)

#result.head(5)


In [41]:
#print((result.text))
stemmed = result.text.to_frame()
print((stemmed))
stemmed['text'] = stemmed['text'].str.replace('[^\w\s]','')
stemmed.to_csv('stemmed.csv',header=False, index=False)

                                                    text
0       worked first week charge phone 20%. waste money.
1      It worked great first couple weeks stopped com...
2      Only works one side time. When connect two cab...
3      bought tried test first ... minutes charging, ...
4      This case reason peeling, much left orginal sk...
...                                                  ...
9997   Better price little milliamps (1500 vs 1440) b...
9998   worked great two friends Nexus 4G's. Fantastic...
9999   My original phone battery losing memory time g...
10000  TOOK ABOUT DAYS TO GET HERE. WORKED AS EXPECTE...
10001  skeptical buying product due mixed reviews peo...

[9996 rows x 1 columns]


In [42]:
stemmed['text'] = stemmed['text'].str.lower()
stemmed.to_csv('stemmed1.csv',header=False, index=False)

In [44]:
NB_WORDS = 10000
tk = Tokenizer(num_words=NB_WORDS,
               filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
               lower=True,
               split=" ")
tk.fit_on_texts(stemmed.text)

X_train_seq = tk.texts_to_sequences(stemmed.text)
#X_test_seq = tk.texts_to_sequences(X_test)

In [22]:
print(X_train_seq[0])

[84, 46, 240, 19, 1, 435, 202, 62]


In [45]:
print(X_train_seq[0])

[80, 47, 233, 19, 2, 431, 192, 58]


In [46]:
X_train_seq_trunc = pad_sequences(X_train_seq, maxlen=80)


In [47]:
print(len(X_train_seq_trunc))

9996


In [48]:
Y_train_seq_trunc = result.rating

In [49]:
print((Y_train_seq_trunc[7777]))

1.0


In [50]:
Y_test = result['rating'].tolist()


In [51]:
print(len(Y_test))

9996


In [52]:
X_test = result['text'].tolist()
#df["new_column"] = df['review'].str.replace('[^\w\s]','')
#print(X_test.head(5))

In [53]:
print(len(Y_test))

9996


In [54]:
X_test = X_train_seq_trunc.tolist()

In [55]:
x_test = np.asarray(X_test)
y_test = np.asarray(Y_test)
y_test = y_test.astype(int)

print('x_test shape:', x_test.shape)

x_test shape: (9996, 80)


In [56]:
print(type(x_test))
print(type(y_test))


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [57]:
print(x_test[0])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  80  47 233  19   2 431 192  58]


In [58]:
print(y_test)


[0 0 0 ... 1 1 1]


In [59]:
print('x_train shape:', x_train.shape)
print(type(x_train))
print(type(x_test))
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)

x_train shape: (25000, 80)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
x_test shape: (9996, 80)
y_test shape: (9996,)


In [60]:
'''
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=2,
          validation_data=(x_test, y_test))
'''

model.fit(x_train, y_train, batch_size=125, epochs=2)


Epoch 1/2
25000/25000 [==============================] - 59s 2ms/step - loss: 0.4886 - acc: 0.7638
Epoch 2/2
25000/25000 [==============================] - 57s 2ms/step - loss: 0.3268 - acc: 0.8646


In [249]:
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)

print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)
print(type(x_test))
print(type(y_test))
print(type(x_train))
print(type(y_train))



x_train shape: (25000, 80)
y_train shape: (25000,)
x_test shape: (9996, 80)
y_test shape: (9996,)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [251]:
results = model.evaluate(x_test, y_test, batch_size=125)
print('test loss, test acc:', results)

9996/9996 [==============================] - 7s 721us/step
test loss, test acc: [1.019766942340405, 0.4681872782938811]


In [61]:
results = model.evaluate(x_test, y_test, batch_size=125)
print('test loss, test acc:', results)

9996/9996 [==============================] - 6s 578us/step
test loss, test acc: [0.9404400996371907, 0.4880952377613185]


In [62]:
results = model.evaluate(x_test, y_test, batch_size=25)
print('test loss, test acc:', results)

9996/9996 [==============================] - 8s 811us/step
test loss, test acc: [0.9404401213986103, 0.488095236996583]
